In [5]:
from src.saving_plan import SavingPlan, get_saving_plans, get_time_periods
from src.visualize import plot_total_worth
import datetime
import polars as pl
from collections import namedtuple

In [6]:
df = pl.read_parquet("data/msci_world_index.parquet").lazy()

In [7]:
start = datetime.datetime(1972, 1, 3) 
end = datetime.datetime(2024, 9, 4)
period = (start, end)
saving_plan = SavingPlan(df=df, invest_amount=10, day_to_invest=22, period=period)

In [8]:
start = datetime.datetime(2022, 2, 3) 
end = datetime.datetime(2024, 7, 4)
period = (start, end)

In [9]:
saving_plans = get_saving_plans(df, 10, period)

In [10]:
total_worths = []
for sp in saving_plans:
    total_worths.append(sp.total_worth)

In [ ]:
plot_total_worth(total_worths, period=period, title="Total worth of 10$ invested on the 22nd of each month")

In [12]:
time_periods = get_time_periods(df, 15)

In [ ]:
from tqdm import tqdm

results = []


In [26]:
simulations = simulate(df, time_periods, 10)